# Práctica SVM
## 1. Idea
Predecir mediante 14 medidas de EEG, si un paciente tiene los ojos abiertos o cerrados, con una precisión mejor al $80 \%$ (para empezar, pues)

Para tal fin se dispone de un dataset donde un paciente abría y cerraba sus ojos durante 117 segundos, con una frecuencia de toma de datos de $128 Hz$

### 1.1 Descripción del dataset

Enlace del dataset: https://archive.ics.uci.edu/ml/datasets/EEG+Eye+State

Enlace de descarga del dataset: https://archive.ics.uci.edu/ml/machine-learning-databases/00264/EEG%20Eye%20State.arff

Instancias = 14980

Atributos:
- @ATTRIBUTE AF3 NUMERIC                                                  
- @ATTRIBUTE F7 NUMERIC                                                   
- @ATTRIBUTE F3 NUMERIC                                                   
- @ATTRIBUTE FC5 NUMERIC                                                  
- @ATTRIBUTE T7 NUMERIC                                                   
- @ATTRIBUTE P7 NUMERIC                                                   
- @ATTRIBUTE O1 NUMERIC                                                   
- @ATTRIBUTE O2 NUMERIC                                                   
- @ATTRIBUTE P8 NUMERIC                                                   
- @ATTRIBUTE T8 NUMERIC                                                   
- @ATTRIBUTE FC6 NUMERIC                                                  
- @ATTRIBUTE F4 NUMERIC                                                   
- @ATTRIBUTE F8 NUMERIC                                                   
- @ATTRIBUTE AF4 NUMERIC                                                  
- @ATTRIBUTE eyeDetection {0,1}

Localización de los sensores:
<img src="eegsensorlocation.jpg">

Cada uno de los atributos corresponde a sensado durante los 117 segundos de medición EEG, y el último atributo es la salida; si los ojos del paciente estaban cerrados (0) o abiertos (1)

## 1.2 Lectura de la información

In [ ]:
import numpy as np
from sklearn.svm import SVC
import matplotlib.pyplot as plt
from time import time
%matplotlib inline

In [ ]:
fileDS = 'EEG Eye State.arff'
data = np.zeros((14980,15))
with open(fileDS, 'rb') as f:
    text = f.readlines()
    prueba = text[19].decode('utf-8')[0:-2].split(',')
    for i, fila in enumerate(text[19::]):
        fila_d = fila.decode('utf-8')[0:-2].split(',')
        data[i] = list(map(float, fila_d))
data_bk = data.copy()

La matriz **data** tiene todos los datos, pero deben seleccionarse solo unos cuantos para el entrenamiento y otros para la prueba.

Para esto entonces se analiza primero el histograma de estado de ojo abierto o cerrado para evitar prejuicios

In [ ]:
X_data = data[::,:-1]
y_data = data[::,-1]

Muestra para los primeros 3 datos:

In [ ]:
print("Datos de los 14 sensores")
print(X_data[0:3])
print("Ojos")
print(y_data[0:3])

Microvoltios, ¿tal vez?

## 1.3 Tratamiento de la información

Se procede a determinar la cantidad de información disponible, por lo que en un histograma podemos ver la frecuencia con la que aparecen los datos de salida (ojos abiertos o cerrados)

In [ ]:
plt.hist(y_data, bins=2)
plt.title("Todos los datos")

Se determina entonces que la mayor cantidad de datos corresponden a ojos cerrados

Para usar $80 \%$ de los datos para el entrenamiento de la SVM y el $20 \%$ restante de los datos para la verificación; $14980*0.8 = 11984$

In [ ]:
plt.hist(y_data[0:11984], bins=2)
plt.title("Primer 80% de los datos")

In [ ]:
plt.hist(y_data[11984::], bins=2)
plt.title("20% restante de los datos")

Los datos de prueba están muy sesgados comparados con los de prueba, por lo que se procede a calcular una selección aleatoria de los datos para entrenamiento, de manera que exista una relación más pareja de la información

In [ ]:
seed = 20 # La semilla se genera para asegurar repetibilidad de la ejecución
np.random.seed(seed)
np.random.shuffle(data) # Esta línea reorganiza aleatoriamente los datos
X_data = data[::,:-1]
y_data = data[::,-1]

Nuevamente se prueban los histogramas

In [ ]:
plt.hist(y_data[0:11984], bins=2)
plt.title("Primer 80% de los datos reorganizados")

In [ ]:
plt.hist(y_data[11984::], bins=2)
plt.title("20% restante de los datos reorganizados")

Están mejor, ahora los dos subconjuntos de datos tienen más o menos la misma relación de estados de ojos abiertos y cerrados

In [ ]:
X_train = X_data[0:11984]
y_train = y_data[0:11984]
X_test = X_data[11984::]
y_test = y_data[11984::]

## 2. Normalización de los datos
Procedimiento llevado a cabo para lidiar con datos que están demasiado desviados

In [ ]:
for i in range(14):
    print("Desviación de", X_train[::,i].std() , "para serie de datos", i+1)

In [ ]:
from sklearn import preprocessing

In [ ]:
scaler = preprocessing.StandardScaler().fit(X_data)

In [ ]:
X_train = scaler.transform(X_train)

In [ ]:
X_test = scaler.transform(X_test)

Ahora la desviación para los datos normalizados es de:

In [ ]:
for i in range(14):
    print("Desviación de", X_train[::,i].std() , "para serie de datos", i+1)

## 3. Creación de la SVM
Se crea la máquina de soporte vectorial y se ajustan los parámetros de la misma

La mejor que he encontrado hasta el momento se da con kernel $rbf$ con: 
- $C=1.2*10^{4}$
- $tol=10^{-7}$
- $\gamma = 1$
- $10^{6}$ de iteraciones

In [ ]:
SVM = SVC()
SVM.set_params(C = 1.2e4, kernel='rbf', tol=1e-7, max_iter = 1e6,
               random_state=None, gamma=1, coef0=0.0, degree=3)
t_start = time()
SVM.fit(X_train, y_train)
t_stop = time()
# Ahora se comprueba con el subconjunto de prueba
errores = np.zeros((1,2996))
for i, Xt in enumerate(X_test):
    errores[0,i] = np.abs(SVM.predict([Xt]) - y_test[i])
t_stoppp = time()
print("Error del:", (errores != 0).sum() / 29.96, "%")
print(t_stop - t_start, "seconds in training")
print(t_stoppp - t_stop, "seconds testing")

Errores con los datos de entrenamiento

In [ ]:
errores = np.zeros((1,11984))
for i, Xt in enumerate(X_train):
    errores[0,i] = np.abs(SVM.predict([Xt]) - y_train[i])
t_stoppp = time()
print("Error del:", (errores != 0).sum() / 119.84, "%")

## 4. Comprobación visual
Ahora para la secuencia, mostrar de manera gráfica la predicción del estado de los ojos y comprararla con los datos.

¡No olvidar normalizar!

In [ ]:
pred = np.zeros((1,14980))
X_pred = data_bk[::,:-1]
X_pred = scaler.transform(X_pred)
for i, Xd in enumerate(X_pred):
    pred[0, i] = SVM.predict([Xd])

In [ ]:
plt.figure(figsize=(10,7))
plt.step(np.arange(0,14980,1)/128, pred[0])
plt.ylim(-0.5, 1.5)

In [ ]:
plt.figure(figsize=(10,7))
plt.step(np.arange(0,14980,1)/128, data_bk[::,-1])
plt.ylim(-0.5, 1.5)